# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra
using ASEconvert

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]
    unit_cell = periodic_system(lattice, atoms, positions)

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE, make the supercell
    # and then convert back ...
    supercell_ase = convert_ase(unit_cell) * pytuple((repeat, 1, 1))
    supercell     = pyconvert(AbstractSystem, supercell_ase)

    # Unfortunately right now the conversion to ASE drops the pseudopotential information,
    # so we need to reattach it:
    supercell = attach_psp(supercell, Al="hgh/lda/al-q3")

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(supercell; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

    CondaPkg Found dependencies: /home/runner/.julia/packages/ASEconvert/CNQ1A/CondaPkg.toml
    CondaPkg Found dependencies: /home/runner/.julia/packages/PythonCall/1f5yE/CondaPkg.toml
    CondaPkg Dependencies already up to date


As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
convert_ase(periodic_system(setup.model)).write("al_supercell.png")

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123


Python: None

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298552764826                   -0.85    5.1         
  2   -8.300223896402       -2.78       -1.25    1.0    110ms
  3   -8.300440236997       -3.66       -1.89    2.1    127ms
  4   -8.300461668172       -4.67       -2.77    3.0    254ms
  5   -8.300464506999       -5.55       -3.19    2.9    157ms
  6   -8.300464583256       -7.12       -3.35    2.0    127ms
  7   -8.300464612424       -7.54       -3.50    1.1    115ms
  8   -8.300464628780       -7.79       -3.64    1.8    146ms
  9   -8.300464638761       -8.00       -3.81    1.8    145ms
 10   -8.300464642122       -8.47       -3.97    1.5    150ms
 11   -8.300464643779       -8.78       -4.27    1.9    142ms


In [4]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -16.64189873320                   -0.70    5.2         
  2   -16.64600614615       -2.39       -1.14    1.6    310ms
  3   -16.67919782654       -1.48       -1.87    2.5    329ms
  4   -16.67927891034       -4.09       -2.67    3.5    395ms
  5   -16.67928595874       -5.15       -3.00    6.9    504ms
  6   -16.67928618465       -6.65       -3.30    1.9    313ms
  7   -16.67928621547       -7.51       -3.74    1.6    287ms
  8   -16.67928622115       -8.25       -4.44    2.6    347ms


In [5]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32650043577                   -0.56    7.1         
  2   -33.33440740434       -2.10       -1.00    1.5    1.16s
  3   -33.33601142488       -2.79       -1.72    6.9    1.66s
  4   -33.33616938437       -3.80       -2.63    5.0    1.50s
  5   -33.33691008706       -3.13       -2.55    7.1    1.86s
  6   -33.33692863239       -4.73       -2.74    2.1    1.10s
  7   -33.33694359164       -4.83       -3.65    2.4    1.30s
  8   -33.33694368127       -7.05       -3.73    4.2    1.49s
  9   -33.33694376053       -7.10       -4.06    1.2    1.01s


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298569620096                   -0.85    5.2         
  2   -8.300274583856       -2.77       -1.59    1.0   88.9ms
  3   -8.300444649080       -3.77       -2.69    2.5    117ms
  4   -8.300448520129       -5.41       -2.65    5.1    215ms
  5   -8.300463970633       -4.81       -3.30    1.0   96.6ms
  6   -8.300464584992       -6.21       -3.77    2.8    138ms
  7   -8.300464641574       -7.25       -4.51    1.5    121ms


In [7]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32373791612                   -0.56    7.0         
  2   -33.30085452319   +   -1.64       -1.26    1.0    914ms
  3   -1.883573507181   +    1.50       -0.32    6.9    2.20s
  4   -33.33181044724        1.50       -1.82    6.4    1.98s
  5   -33.25581308118   +   -1.12       -1.36    4.0    1.72s
  6   -33.32150120781       -1.18       -1.82    2.8    1.41s
  7   -33.08094733348   +   -0.62       -1.36    4.5    1.48s
  8   -33.31909696718       -0.62       -1.91    5.0    1.50s
  9   -33.33614309543       -1.77       -2.29    2.6    1.14s
 10   -33.33668935784       -3.26       -2.75    2.4    1.21s
 11   -33.33689314583       -3.69       -3.01    3.4    1.37s
 12   -33.33690354894       -4.98      

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.